In [2]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
#import  plotly.offline  as py
import plotly.graph_objs as go
from plotly import tools
import plotly.graph_objs as graph_objs
import json
import os
from mapboxgl.viz import *
from mapboxgl.utils import *
from colour import Color

%matplotlib inline
#py.init_notebook_mode(connected=True)

In [3]:
ds_2018 = pd.read_excel('total.xls')
ds_2018 = ds_2018.set_index('Code')
ds_2018 = ds_2018.drop('Départements', axis=1)
ds_2018.head()
ds_2010 = pd.read_excel('total_2010.xls')
ds_2010 = ds_2010.set_index('Code')
ds_2010 = ds_2010.drop('Départements', axis=1)

ds_2010.head()

,Spécialistes par qualification - ANATOMIE ET CYTOLOGIE-PATHOLOGIQUES - 2010,Spécialistes par qualification - ANESTHESIE-REANIMATION - 2010,Spécialistes par qualification - BIOLOGIE MEDICALE - 2010,Spécialistes par qualification - CARDIOLOGIE ET MALADIES VASCULAIRES - 2010,Spécialistes par qualification - CHIRURGIE GENERALE - 2010,Spécialistes par qualification - CHIRURGIE MAXILLO-FACIALE - 2010,Spécialistes par qualification - CHIRURGIE ORTHOPEDIQUE ET TRAUMATOLOGIE - 2010,"Spécialistes par qualification - CHIRURGIE PLASTIQUE, RECONSTRUCTRICE ET ESTHETIQUE - 2010",Spécialistes par qualification - CHIRURGIE THORACIQUE ET CARDIO-VASCULAIRE - 2010,Spécialistes par qualification - CHIRURGIE UROLOGIQUE - 2010,...,Spécialistes par qualification - RHUMATOLOGIE - 2010,Spécialistes par qualification - SANTE PUBLIQUE ET MEDECINE SOCIALE - 2010,Spécialistes par qualification - STOMATOLOGIE - 2010,Spécialistes par qualification - CHIRURGIE DE LA FACE ET DU COU - 2010,Spécialistes par qualification - CHIRURGIE INFANTILE - 2010,Spécialistes par qualification - CHIRURGIE ORALE - 2010,Spécialistes par qualification - NEURO-PSYCHIATRIE - 2010,Spécialistes par qualification - ONCO-HEMATOLOGIE - 2010,Spécialistes par qualification - RADIODIAGNOSTIC ET RADIOTHERAPIE - 2010,Spécialistes par qualification - REANIMATION MEDICALE - 2010
Code,,,,,,,,,,,,,,,,,,,,,
01,4.0,34,7.0,17,10,NaN,14,1.0,NaN,3.0,...,12,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02,3.0,38,11.0,30,21,NaN,24,1.0,NaN,5.0,...,15,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
03,3.0,35,6.0,22,9,1.0,16,3.0,NaN,9.0,...,14,3.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,3.0
04,2.0,14,5.0,11,4,NaN,7,NaN,NaN,1.0,...,4,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05,3.0,28,5.0,12,6,NaN,7,NaN,1.0,4.0,...,3,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
ds_2018.columns = list(map(lambda x: x.replace('Spécialistes par qualification - ', ''), ds_2018.columns))
ds_2018.columns = list(map(lambda x: x.replace(' - 2018', ''), ds_2018.columns))
ds_2018.columns
ds_2010.columns = list(map(lambda x: x.replace('Spécialistes par qualification - ', ''), ds_2010.columns))
ds_2010.columns = list(map(lambda x: x.replace(' - 2010', ''), ds_2010.columns))
sorted(list(ds_2010.columns))

['ANATOMIE ET CYTOLOGIE-PATHOLOGIQUES',
 'ANESTHESIE-REANIMATION',
 'BIOLOGIE MEDICALE',
 'CARDIOLOGIE ET MALADIES VASCULAIRES',
 'CHIRURGIE DE LA FACE ET DU COU',
 'CHIRURGIE GENERALE',
 'CHIRURGIE INFANTILE',
 'CHIRURGIE MAXILLO-FACIALE',
 'CHIRURGIE ORALE',
 'CHIRURGIE ORTHOPEDIQUE ET TRAUMATOLOGIE',
 'CHIRURGIE PLASTIQUE, RECONSTRUCTRICE ET ESTHETIQUE',
 'CHIRURGIE THORACIQUE ET CARDIO-VASCULAIRE',
 'CHIRURGIE UROLOGIQUE',
 'CHIRURGIE VASCULAIRE',
 'CHIRURGIE VISCERALE ET DIGESTIVE',
 'DERMATOLOGIE ET VENEREOLOGIE',
 'ENDOCRINOLOGIE ET METABOLISME',
 'GASTRO-ENTEROLOGIE ET HEPATOLOGIE',
 'GENETIQUE MEDICALE',
 'GERIATRIE',
 'GYNECOLOGIE MEDICALE',
 'GYNECOLOGIE MEDICALE ET OBSTETRIQUE - 2018',
 'GYNECOLOGIE-OBSTETRIQUE',
 'HEMATOLOGIE',
 'MEDECINE DU TRAVAIL',
 'MEDECINE GENERALE',
 'MEDECINE INTERNE',
 'MEDECINE NUCLEAIRE',
 'MEDECINE PHYSIQUE ET DE READAPTATION',
 'NEPHROLOGIE',
 'NEURO-PSYCHIATRIE',
 'NEUROCHIRURGIE',
 'NEUROLOGIE',
 'ONCO-HEMATOLOGIE',
 'ONCOLOGIE MEDICALE',
 '

In [5]:
ds_2018.head()
sum_2018 = ds_2018.sum()
sum_2010 = ds_2010.sum()
sum_2018.head(15)

ANATOMIE ET CYTOLOGIE-PATHOLOGIQUES                    1522.0
ANESTHESIE-REANIMATION                                10066.0
BIOLOGIE MEDICALE                                      2894.0
CARDIOLOGIE ET MALADIES VASCULAIRES                    6209.0
CHIRURGIE GENERALE                                     3134.0
CHIRURGIE ORTHOPEDIQUE ET TRAUMATOLOGIE                3199.0
CHIRURGIE PLASTIQUE, RECONSTRUCTRICE ET ESTHETIQUE      873.0
CHIRURGIE THORACIQUE ET CARDIO-VASCULAIRE               462.0
CHIRURGIE UROLOGIQUE                                   1243.0
CHIRURGIE VASCULAIRE                                    493.0
CHIRURGIE VISCERALE ET DIGESTIVE                       1020.0
DERMATOLOGIE ET VENEREOLOGIE                           3316.0
ENDOCRINOLOGIE ET METABOLISME                          1832.0
GASTRO-ENTEROLOGIE ET HEPATOLOGIE                      3504.0
GENETIQUE MEDICALE                                      247.0
dtype: float64

In [6]:
result = pd.concat([sum_2010, sum_2018], axis=1).dropna()
result = result.drop('ONCO-HEMATOLOGIE')
result['diff'] = ((result[1] - result[0]) / result[0])*100

In [9]:
palette = list(Color('red').range_to(Color('green'), len(result)))
unique = list(result['diff'].sort_values().unique())
result['color'] = result['diff'].apply(lambda x: palette[unique.index(x)].hex)

In [10]:
layout = go.Layout(
    xaxis=dict(
        type='log',
        autorange=True,
        dtick=2,
        title='Nombre de practiciens actuellement (2018)'
    ),
    yaxis=dict(
        type='linear',
        autorange=True,
        dtick=10,
        ticksuffix='%',
        tickmode='array',
        tickvals=[-100, -75, -50, -20, 20, 50, 100, 150, 200, 400],
        title='Évolution de 2010 à 2018'
    ),
    hovermode='closest',
)
text = []
for item in result.iterrows():
    text.append("{name}<br>Evolution:{evolution:.2f}%<br>Total:{total}".format(name=item[0], evolution=item[1][2], total=int(item[1][1]))) 

points = go.Scatter(
        y=result['diff'],
        x=result[1],
        mode='markers',
        marker=dict(
            size=result[1]/np.mean(result[1]) * 40,
            sizemode='area',
            sizeref=2.*100/(40.**2),
            sizemin=4,
            color=result['color'],            #[[-100.0, 'rgb(165,0,38)'], [0.0, 'rgb(165,0,38)'], [100.0, 'rgb(165,0,38)']],
        ),
        hoverinfo="text",
        text=text,
)
data = [points]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='total_practiciens')